# Strands Agent에서 Model Context Protocol (MCP)을 도구로 사용하기

## 개요
[Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction)은 애플리케이션이 대규모 언어 모델(LLM)에 컨텍스트를 제공하는 방법을 표준화하는 개방형 프로토콜입니다. Strands AI SDK는 MCP와 통합되어 외부 도구 및 서비스를 통해 에이전트 기능을 확장합니다.

MCP는 추가 도구를 제공하는 MCP 서버와 에이전트 간의 통신을 가능하게 합니다. Strands Agent SDK에는 MCP 서버에 연결하고 해당 도구를 사용하기 위한 기본 제공 지원이 포함되어 있습니다.

이 예제에서는 Strands Agent에서 MCP 도구를 사용하는 방법을 보여드리겠습니다. AWS 문서에 액세스하고, 콘텐츠를 검색하고, 권장 사항을 얻을 수 있는 도구를 제공하는 [AWS Documentation MCP server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/)를 사용합니다. 이 MCP 서버에는 3가지 주요 기능이 있습니다:

- **문서 읽기**: AWS 문서 페이지를 가져와서 마크다운 형식으로 변환
- **문서 검색**: 공식 검색 API를 사용하여 AWS 문서 검색
- **권장 사항**: AWS 문서 페이지에 대한 콘텐츠 권장 사항 가져오기



## 에이전트 세부 정보
<div style="float: left; margin-right: 20px;">
    
|기능                |설명                                               |
|--------------------|--------------------------------------------------|
|사용된 기능         |MCP Tools                                         |
|에이전트 구조       |단일 에이전트 아키텍처                               |

</div>

## 아키텍처

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

## 주요 기능
* **단일 에이전트 아키텍처**: 이 예제는 MCP 도구와 상호 작용하는 단일 에이전트를 생성합니다
* **MCP 도구**: MCP 도구와 에이전트의 통합

## 설정 및 사전 요구사항

### 사전 요구사항
* Python 3.10+
* AWS 계정
* Amazon Bedrock에서 활성화된 Anthropic Claude 3.7

이제 Strands Agent에 필요한 패키지를 설치하겠습니다

In [1]:
# installing pre-requisites
!pip install -r requirements.txt

### 종속성 패키지 가져오기

이제 종속성 패키지를 가져오겠습니다

In [2]:
import threading
import time
from datetime import timedelta

from mcp import StdioServerParameters, stdio_client
from mcp.client.streamable_http import streamablehttp_client
from mcp.server import FastMCP
from strands import Agent
from strands.tools.mcp import MCPClient

### stdio 전송을 사용하여 MCP 서버에 연결

MCP의 [Transports](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports)는 클라이언트와 서버 간의 통신을 위한 기반을 제공합니다. 메시지가 전송 및 수신되는 방법의 기본 메커니즘을 처리합니다. 현재 MCP에 내장된 세 가지 표준 전송 구현이 있습니다:

- **Standard Input/Output (stdio)**: 표준 입력 및 출력 스트림을 통한 통신을 가능하게 합니다. 로컬 통합 및 명령줄 도구에 특히 유용합니다
- **Streamable HTTP**: 이전 프로토콜 버전의 HTTP+SSE 전송을 대체합니다. Streamable HTTP 전송에서 서버는 여러 클라이언트 연결을 처리할 수 있는 독립적인 프로세스로 작동합니다. 이 전송은 HTTP POST 및 GET 요청을 사용합니다. 서버는 선택적으로 Server-Sent Events (SSE)를 사용하여 여러 서버 메시지를 스트리밍할 수 있습니다. 이를 통해 기본 MCP 서버는 물론 스트리밍 및 서버-클라이언트 알림 및 요청을 지원하는 더 많은 기능을 갖춘 서버를 사용할 수 있습니다.
- **SSE**: Server-Sent Events 전송을 사용하는 HTTP 기반 MCP 서버를 위한 레거시 전송

전반적으로 명령줄 도구 구축, 로컬 통합 구현 및 셸 스크립트 작업에는 stdio를 사용해야 합니다. AI 에이전트가 도구 및 서비스와 통신하는 유연하고 효율적인 방법이 필요한 경우, 특히 상태 비저장 통신을 처리하거나 리소스 사용을 최소화하는 것이 중요한 경우 Streamable HTTP 전송을 사용해야 합니다.

특정 요구 사항에 맞는 **커스텀 전송** 구현을 사용할 수도 있습니다.


이제 stdio 전송을 사용하여 MCP 서버에 연결하겠습니다. 먼저 `MCPClient` 클래스를 사용하여 [AWS Documentation MCP Server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/)에 연결합니다. 이 서버는 AWS 문서에 액세스하고, 콘텐츠를 검색하고, 권장 사항을 얻을 수 있는 도구를 제공합니다.

In [3]:
# Connect to an MCP server using stdio transport
stdio_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

#### 에이전트 구성 설정 및 호출

다음으로 방금 생성한 `stdio_mcp_client` 객체의 도구를 사용하여 에이전트 구성을 설정합니다. 이를 위해 MCP 서버에서 사용 가능한 도구를 나열해야 합니다. 이를 위해 `list_tools_sync` 메서드를 사용할 수 있습니다.

그런 다음 에이전트에 질문을 합니다.

In [5]:
# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent("Amazon Bedrock 가격 모델이란 무엇인가요? 간결하게 설명해 주세요.")

Amazon Bedrock의 가격 모델에 대한 정보를 찾아보겠습니다. AWS 공식 문서에서 이 정보를 검색하겠습니다.
Tool #1: search_documentation
첫 번째 결과가 Amazon Bedrock의 가격 책정에 대한 문서인 것 같습니다. 이 문서를 확인해 보겠습니다.
Tool #2: read_documentation
Amazon Bedrock의 가격 책정에 대한 정보를 더 자세히 알아보기 위해 Provisioned Throughput에 대한 내용도 확인해 보겠습니다.
Tool #3: read_documentation
이제 Amazon Bedrock의 가격 모델에 대한 충분한 정보를 확보했습니다. 간결하게 설명해 드리겠습니다.

## Amazon Bedrock 가격 모델

Amazon Bedrock의 가격 모델은 다음과 같이 요약됩니다:

1. **온디맨드 요금제**: 입력 토큰(input tokens)과 출력 토큰(output tokens)의 양에 따라 비용이 청구됩니다. 사용한 만큼만 지불하는 방식입니다.

2. **Provisioned Throughput(프로비저닝된 처리량)**: 고정 비용으로 일정 수준의 처리량을 확보할 수 있습니다.
   - 약정 기간에 따라 할인율이 달라집니다 (무약정, 1개월 약정, 6개월 약정)
   - Model Units(MU)라는 단위로 처리량을 지정하며, 이는 모델별로 분당 처리 가능한 입력 및 출력 토큰 수를 정의합니다.
   - 기본 모델 및 사용자 지정 모델 모두에 적용 가능합니다.

Amazon Bedrock을 사용하면 기본적으로 모델 추론 실행에 대해서만 비용을 지불하며, 사용량 또는 확보한 처리량에 따라 요금이 책정됩니다.

### Streamable HTTP를 사용하여 MCP 서버에 연결

이제 Streamable HTTP 전송을 사용하여 MCP 서버에 연결하겠습니다. 먼저 Streamable HTTP 전송을 사용하여 간단한 MCP 서버를 시작하겠습니다.

이 예제에서는 자체 MCP 서버를 생성합니다. 아키텍처는 다음과 같습니다

<div style="text-align:center">
    <img src="images/architecture_2.png" width="65%" />
</div>

In [6]:
# Create an MCP server
mcp = FastMCP("Calculator Server")

# Define a tool


@mcp.tool(description="Calculator tool which performs calculations")
def calculator(x: int, y: int) -> int:
    return x + y


@mcp.tool(description="This is a long running tool")
def long_running_tool(name: str) -> str:
    time.sleep(25)
    return f"Hello {name}"


def main():
    mcp.run(transport="streamable-http", mount_path="mcp")

이제 `streamable-http` 서버로 스레드를 시작하겠습니다

In [7]:
thread = threading.Thread(target=main)
thread.start()

INFO:     Started server process [11067]
INFO:     Waiting for application startup.


[10/01/25 23:47:39] INFO     StreamableHTTP session manager started                  ]8;id=955636;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=605153;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#110\110]8;;\

INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:34118 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:34136 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:34134 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:34148 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41132 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41146 - "DELETE /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51102 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51120 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51116 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:51122 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:35434 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:35438 - "DELETE /mcp HTTP/1.1" 200 OK


#### Streamable HTTP 클라이언트와 Agent 통합

이제 `streamablehttp_client`를 사용하여 이 서버를 간단한 에이전트와 통합하겠습니다.

In [8]:
def create_streamable_http_transport():
    return streamablehttp_client("http://localhost:8000/mcp")


streamable_http_mcp_client = MCPClient(create_streamable_http_transport)

#### 에이전트 구성 설정 및 호출

다음으로 방금 생성한 `streamable_http_mcp_client` 객체의 도구를 사용하여 에이전트 구성을 설정합니다. 이를 위해 MCP 서버에서 사용 가능한 도구를 나열해야 합니다. 이를 위해 `list_tools_sync` 메서드를 사용할 수 있습니다.

그런 다음 에이전트에 질문을 합니다.

In [9]:
with streamable_http_mcp_client:
    tools = streamable_http_mcp_client.list_tools_sync()

    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = str(agent("What is 2 + 2?"))

[10/01/25 23:47:52] INFO     Created new transport with session ID:                  ]8;id=919634;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=120855;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             5be482fa20a446eabb34c26879f8ad7b                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=959246;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=945802;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 5be482fa20a446eabb34c26879f8ad7b           ]8;id=891797;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=8818;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=980393;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=573440;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#146\146]8;;\

                    INFO     HTTP Request: GET http://localhost:8000/mcp "HTTP/1.1 200 OK"          ]8;id=635160;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=204111;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

...

[10/01/25 23:47:55] INFO     Processing request of type CallToolRequest                               ]8;id=133384;file:///opt/conda/lib/python3.12/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=182753;file:///opt/conda/lib/python3.12/site-packages/mcp/server/lowlevel/server.py#624\624]8;;\

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=926450;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=311763;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

The answer to 2 + 2 is 4.

[10/01/25 23:47:56] INFO     Terminating session: 5be482fa20a446eabb34c26879f8ad7b           ]8;id=708792;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=331637;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py#630\630]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=895195;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=24498;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

### 직접 도구 호출

도구는 일반적으로 사용자 요청에 따라 에이전트에 의해 호출되지만 MCP 도구를 직접 호출할 수도 있습니다. 이는 여러 도구를 함께 오케스트레이션하는 워크플로 시나리오에 유용할 수 있습니다.

In [ ]:
query = {"x": 10, "y": 20}

with streamable_http_mcp_client:
    # direct tool invocation
    result = streamable_http_mcp_client.call_tool_sync(
        tool_use_id="tool-123", name="calculator", arguments=query
    )

    # Process the result
    print(f"Calculation result: {result['content'][0]['text']}")

MCP 서버 도구를 호출할 때 너무 오래 실행되지 않도록 선택적으로 `read_timeout_seconds`를 제공할 수도 있습니다

In [10]:
with streamable_http_mcp_client:
    try:
        result = streamable_http_mcp_client.call_tool_sync(
            tool_use_id="tool-123",
            name="long_running_tool",
            arguments={"name": "Amazon"},
            read_timeout_seconds=timedelta(seconds=30),
        )

        if result["status"] == "error":
            print(f"Tool execution failed: {result['content'][0]['text']}")
        else:
            print(f"Tool execution succeeded: {result['content'][0]['text']}")
    except Exception as e:
        print(f"Tool call timed out or failed: {str(e)}")

[10/01/25 23:48:13] INFO     Created new transport with session ID:                  ]8;id=949821;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=60486;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             6c32966eb15a4256b6665eab014ee446                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=552614;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=459041;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 6c32966eb15a4256b6665eab014ee446           ]8;id=280592;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=210323;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=399896;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=422585;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#146\146]8;;\

                    INFO     HTTP Request: GET http://localhost:8000/mcp "HTTP/1.1 200 OK"          ]8;id=1255;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=161951;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

...

[10/01/25 23:48:38] INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=915412;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=730043;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Processing request of type ListToolsRequest                              ]8;id=985498;file:///opt/conda/lib/python3.12/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=767147;file:///opt/conda/lib/python3.12/site-packages/mcp/server/lowlevel/server.py#624\624]8;;\

Tool execution succeeded: Hello Amazon


                    INFO     Terminating session: 6c32966eb15a4256b6665eab014ee446           ]8;id=966928;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=775955;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py#630\630]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=681901;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=460938;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

### 여러 MCP 서버와 상호 작용

Strands Agents를 사용하면 동일한 에이전트를 사용하여 여러 MCP 서버와 상호 작용하고 병렬로 사용할 수 있는 최대 도구 수(`max_parallel_tools`)와 같은 도구 설정을 구성할 수도 있습니다. 이 구성을 보여주기 위해 새 에이전트를 만들어 보겠습니다:

<div style="text-align:center">
    <img src="images/architecture_3.png" width="85%" />
</div>

이 에이전트에서는 AWS Documentation MCP 서버를 다시 사용하고 AWS Cloud Development Kit (CDK) 모범 사례, 코드형 인프라 패턴 및 CDK Nag를 사용한 보안 규정 준수를 지원하는 [AWS CDK MCP Server](https://awslabs.github.io/mcp/servers/cdk-mcp-server/)도 사용합니다.

먼저 stdio 전송을 사용하여 두 MCP 서버에 연결하겠습니다

In [12]:
# Connect to an MCP server using stdio transport
aws_docs_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

# Connect to an MCP server using stdio transport
cdk_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(command="uvx", args=["awslabs.cdk-mcp-server@latest"])
    )
)

#### MCP 서버로 에이전트 생성

다음으로 두 MCP 서버의 도구를 사용하여 에이전트를 생성합니다

In [13]:
# Create an agent with MCP tools
with aws_docs_mcp_client, cdk_mcp_client:
    # Get the tools from the MCP server
    tools = aws_docs_mcp_client.list_tools_sync() + cdk_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent(
        "Amazon Bedrock의 가격 모델은 무엇인가요? 간결하게 설명해 주세요. 또한 CDK와 관련된 모범 사례는 무엇인가요??"
    )

질문을 두 부분으로 나누어 답변하겠습니다.

먼저, Amazon Bedrock의 가격 모델에 대한 정보를 찾아보겠습니다.
Tool #1: search_documentation
가격에 대한 자세한 정보를 확인하기 위해 첫 번째 결과 페이지를 읽어보겠습니다.
Tool #2: read_documentation
다음으로 CDK와 관련된 모범 사례에 대한 정보를 찾아보겠습니다.
Tool #3: CDKGeneralGuidance
제공된 정보를 바탕으로 Amazon Bedrock의 가격 모델과 CDK 관련 모범 사례에 대해 간결하게 답변드리겠습니다:

## Amazon Bedrock 가격 모델

Amazon Bedrock의 가격 모델은 다음과 같습니다:

1. **사용량 기반 과금**: 기반 모델(foundation models)에 대한 추론(inference) 실행에 비용을 지불합니다.
2. **토큰 기반 가격 책정**: 입력 토큰과 출력 토큰의 양에 따라 가격이 결정됩니다.
3. **프로비저닝된 처리량(Provisioned Throughput)**: 모델에 대해 프로비저닝된 처리량을 구매했는지에 따라 가격이 달라집니다.
4. **모델별 가격**: 각 모델 제공업체와 모델 버전에 따라 가격이 다르게 책정됩니다.

자세한 가격 정보는 [Amazon Bedrock 가격 책정 페이지](https://aws.amazon.com/bedrock/pricing)에서 확인할 수 있습니다.

## CDK 관련 모범 사례

AWS CDK 사용 시 주요 모범 사례는 다음과 같습니다:

1. **적절한 초기화**: CDK CLI를 사용하여 프로젝트를 올바르게 초기화합니다.
   ```bash
   cdk init app --language typescript  # TypeScript 프로젝트의 경우
   cdk init app --language python  # Python 프로젝트의 경우
   ```

2. **효율적인 개발 워크플로**:
   - `cdk synth`: CloudForm

### 축하합니다!

이 노트북에서는 Strands Agent를 사용하여 MCP 서버에 연결하는 방법과 두 가지 MCP 전송 프로토콜인 stdio 및 Streamable HTTP를 배웠습니다. 또한 여러 MCP 서버를 동일한 에이전트에 연결하는 방법도 배웠습니다. 다음으로 에이전트에서 다양한 모델을 사용하는 방법을 살펴보겠습니다